In [1]:
import random
import os, sys
from importlib import import_module
from PIL import Image
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from torch.utils.data import Subset
from torch.optim import SGD, Adam, AdamW, RMSprop
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
from glob import glob
sys.path.append(os.path.abspath('..'))
# from model import *
import timm
import time
from adamp import AdamP
from collections import defaultdict
import wandb

def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정
    
    Args:
        seed : seed 정수 값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(17)

In [2]:
img_root = 'input/data/train/images'
label_path = 'input/data/train/train.csv'

model_name = 'efficientnet_b3_pruned'
use_pretrained = True
freeze_backbone = False # classifier head 이 외 부분을 업데이트되지 않게 할 것인지 여부

val_split = 0.2
batch_size = 64
num_workers = 3
num_classes = 6

num_epochs = 5
lr = 1e-4
lr_decay_step = 10

train_log_interval = 20

# -- settings
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [3]:
class FocalLoss(nn.Module):
    def __init__(self, weight = None, gamma = 2, reduction = 'mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim = -1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1-prob)**self.gamma) * log_prob, 
            target_tensor,
            weight = self.weight,
            reduction = self.reduction
        )
    
    

In [4]:
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, 2).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        return 1 - f1.mean()

In [5]:
class Focal_F1(nn.Module):
    def __init__(self, weight = None, gamma = 2, reduction = 'mean', epsilon=1e-7):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
#         def __init__(self, epsilon=1e-7):
#         super().__init__()
        self.epsilon = epsilon
    # def forward(self, input_tensor, target_tensor):
        
    def forward(self, input_tensor, target_tensor,):
        assert input_tensor.ndim == 2
        assert target_tensor.ndim == 1
        target_tensor = F.one_hot(target_tensor, 2).to(torch.float32)
        input_tensor = F.softmax(input_tensor, dim=1)
        
        tp = (target_tensor * input_tensor).sum(dim=0).to(torch.float32)
        tn = ((1 - target_tensor) * (1 - input_tensor)).sum(dim=0).to(torch.float32)
        fp = ((1 - target_tensor) * input_tensor).sum(dim=0).to(torch.float32)
        fn = (target_tensor * (1 - input_tensor)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        # return 1 - f1.mean()
        log_prob = F.log_softmax(input_tensor, dim = -1)
        prob = torch.exp(log_prob)
        
        
        return F.nll_loss(
            ((1-prob)**self.gamma) * log_prob, 
            target_tensor,
            weight = self.weight,
            reduction = self.reduction
        ) + 1 - f1.mean()

In [6]:
criterion = FocalLoss()

In [7]:
# model = timm.create_model(model_name,pretrained = True)
# # model
# # last_num  = model.classifier.in_features
# # model.classifier = nn.Linear(last_num,num_classes,bias=True)
# model.reset_classifier(num_classes)
# model.to(device)

# optimizer = AdamP(model.parameters(), lr = lr, weight_decay = 5e-4)

# scheduler = CosineAnnealingLR(optimizer, T_max = 2, eta_min = 0.)

In [8]:
### Configuration
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train.csv'
df = pd.read_csv(df_path)

# 한 디렉토리 안에 여러 확장자가 들어있는 경우에 맞게 수정
def get_ext(img_dir,img_id):
    filename = glob(os.path.join(img_dir,img_id)+'/*.*')
#     print(filename)
    ext = list(map(lambda x : os.path.splitext(x)[-1].lower(), filename))
    return ext


In [9]:
from albumentations import *
from albumentations.pytorch import ToTensorV2

# def get_transforms(need = ('train','val'), img_size = (512,384), mean = (0.560, .524,.501), std = (.233,.243,.247)):
def get_transforms(need = ('train','val'), img_size = (300,300), mean = (0.560, .524,.501), std = (.233,.243,.247)):
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            CenterCrop(500,300, p = 1.),
            HorizontalFlip(p = 0.5),
            RandomBrightnessContrast(brightness_limit = (-0.1,0.1), contrast_limit = (-0.1,0.1), p = 0.5),
            Normalize(mean=mean, std=std, max_pixel_value = 255., p = 1.),
            ToTensorV2(p = 1.),
        ], p = 1.)
        
    if 'val' in need :
        transformations['val'] = Compose([
            CenterCrop(500,300, p = 1.),
            Normalize(mean = mean, std=std, max_pixel_value = 255. , p = 1.),
            ToTensorV2(p = 1.),
        ], p = 1.)
    return transformations

In [10]:
class MaskLabels:
    mask = 0
    incorrect = 1
    normal = 2
    
class GenderLabels:
    male = 0
    female = 1
    
class AgeGroup:
    map_label = lambda x : 0 if int(x) < 30 else 1 if int(x) < 58 else 2

In [11]:
class MaskBaseDataset(data.Dataset):
    num_classes = 3*2
#     num_classes = 3*2*3
    
    _file_names = {
        'mask1' : MaskLabels.mask,
        'mask2' : MaskLabels.mask,
        'mask3' : MaskLabels.mask,
        'mask4' : MaskLabels.mask,
        'mask5' : MaskLabels.mask,
        'incorrect_mask' : MaskLabels.incorrect,
        'normal' : MaskLabels.normal
    }
    
    image_paths = []
    mask_labels = []
    gender_labels =[]
    age_labels = []
    multi_labels = []
    def __init__(self, img_dir, transform = None):
        self.img_dir = img_dir
        self.mean = mean
        self.std = std
        self.transform = transform
        
        self.setup()
        
    def set_transform(self, transform):
        self.transform = transform
    
    def setup(self):
        profiles = glob(self.img_dir+"/*")
        for profile in profiles:
            for file_name, label in zip(list(map(lambda x: x[0]+x[1], zip(self._file_names.keys(), get_ext("",profile)))),self._file_names.values()):
                img_path = os.path.join(self.img_dir, profile, file_name)
                if os.path.exists(img_path):
                    self.image_paths.append(img_path)
                    id, gender, race, age = profile.split('_')
                    gender_label = getattr(GenderLabels,gender)
                    age_label = AgeGroup.map_label(age)
                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)
                    self.multi_labels.append(gender_label * 3 + age_label)
    def __getitem__(self, index, item = None):
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        gender_label = self.gender_labels[index]
        age_label = self.age_labels[index]
#         multi_class_label = gender_label * 3 + age_label
        multi_class_label = self.multi_labels[index]
        image_transform = self.transform(image=np.array(image))['image']
        return image_transform, multi_class_label
    
    def __len__(self):
        return len(self.image_paths)

In [12]:
mean = (0.560, .524,.501)
std = (.233,.243,.247)
transform = get_transforms(mean = mean, std = std)
    
dataset = MaskBaseDataset(img_dir = img_dir)

# n_val = int(len(dataset) * val_split)
# n_train = len(dataset) - n_val
# train_dataset, val_dataset = data.random_split(dataset, [n_train, n_val])

# train_dataset.dataset.set_transform(transform['train'])
# val_dataset.dataset.set_transform(transform['val'])

In [13]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits =5)

# stk = StratifiedKFold(n_splits=num_epochs, shuffle=False)
# for fold, x in enumerate(stk.split(dataset.image_paths, dataset.age_gender_labels),1):
for fold, (train_idx, val_idx) in enumerate(skf.split(dataset.image_paths, dataset.multi_labels),1):
#     print(fold,x)
    train_dataset = data.Subset(dataset, train_idx)
    val_dataset = data.Subset(dataset,val_idx)
    train_dataset.dataset.set_transform(transform['train'])
    val_dataset.dataset.set_transform(transform['val'])

    train_loader = data.DataLoader(
        train_dataset,
        batch_size = batch_size,
        num_workers = num_workers,
        shuffle = True
    )

    val_loader = data.DataLoader(
        val_dataset,
        batch_size = batch_size,
        num_workers = num_workers,
        shuffle = False
    )
    
    model = timm.create_model(model_name,pretrained = True)
# model
    # last_num  = model.classifier.in_features
    # model.classifier = nn.Linear(last_num,num_classes,bias=True)
    model.reset_classifier(num_classes)
    model.to(device)

    optimizer = AdamP(model.parameters(), lr = lr, weight_decay = 5e-4)

    scheduler = CosineAnnealingLR(optimizer, T_max = 2, eta_min = 0.)
    patience = 10
    counter = 0

    accumulation_steps = 2



    os.makedirs(os.path.join(os.getcwd(),'results2', model_name), exist_ok = True)
    wandb.init(config= {
            "batch_size" : batch_size,
            "lr": lr,
            "epochs": num_epochs,
            "backbone": 'efficientnet_b3_pruned'

        })
    counter =0 
    best_val_acc = 0
    best_val_loss = np.inf
    for epoch in range(num_epochs):
        #train loop
        model.train()
        loss_value = 0
        matches = 0
        for idx, train_batch in enumerate(train_loader):
            inputs, labels= train_batch
            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim = -1)
            loss = criterion(outs, labels)

            loss.backward()

            if (idx+1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            loss_value += loss.item()
            matches += (preds == labels).sum().item()
            if (idx+1) % train_log_interval == 0:
                train_loss = loss_value/ train_log_interval
                train_acc = matches / batch_size/ train_log_interval
                current_lr = scheduler.get_last_lr()
                wandb.log({
                        "Train_loss": train_loss,
                        "Train_acc": train_acc
                    })
                print(
                    f'Epoch[{epoch}/{num_epochs}]({idx+1}/{len(train_loader)})||'
                    f'training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}'
                )

                loss_value = 0
                matches = 0
        scheduler.step()

        #val loop
        with torch.no_grad():
            print("Calculating validation results")
            model.eval()
            val_loss_items = []
            val_acc_items= []
            for val_batch in val_loader:
                inputs, labels = val_batch
                inputs = inputs.to(device)
                labels = labels.to(device)

                outs = model(inputs)
                preds = torch.argmax(outs, dim = -1)

                loss_item = criterion(outs, labels).item()
                acc_item = (labels == preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)

            val_loss = np.sum(val_loss_items)/ len(val_loader)
            val_acc  = np.sum(val_acc_items)/ len(val_dataset)
            wandb.log({
                    "Valid_loss": val_loss,
                    "Valid_acc": val_acc
                })
            if val_loss < best_val_loss:
                best_val_loss = val_loss
            if val_acc > best_val_acc:
                print('New best model for val accuracy! saving the model..')
                now = time.localtime()
    # print(now.tm_hour+9)
                torch.save(model.state_dict(), f'results2/{model_name}/{fold}fold_{epoch:03}_accuracy_{val_acc:4.2%}_{now.tm_hour+9}:{now.tm_min}_agegender.ckpt')
                best_val_acc = val_acc
                counter = 0
            else:
                counter += 1

            if counter > patience:
                print('Early stopping...')
                break

            print(
                f'[Val] acc : {val_acc:4.2%}, loss : {val_loss:4.2} ||'
                f'best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}'
            )

wandb: Currently logged in as: hwan17 (use `wandb login --relogin` to force relogin)


Epoch[0/5](20/237)||training loss 0.8015 || training accuracy 62.81% || lr [0.0001]
Epoch[0/5](40/237)||training loss 0.3036 || training accuracy 79.06% || lr [0.0001]
Epoch[0/5](60/237)||training loss 0.1584 || training accuracy 83.36% || lr [0.0001]
Epoch[0/5](80/237)||training loss 0.1628 || training accuracy 83.83% || lr [0.0001]
Epoch[0/5](100/237)||training loss 0.1078 || training accuracy 87.11% || lr [0.0001]
Epoch[0/5](120/237)||training loss 0.1063 || training accuracy 87.27% || lr [0.0001]
Epoch[0/5](140/237)||training loss 0.1013 || training accuracy 86.95% || lr [0.0001]
Epoch[0/5](160/237)||training loss 0.08308 || training accuracy 89.77% || lr [0.0001]
Epoch[0/5](180/237)||training loss 0.08073 || training accuracy 88.75% || lr [0.0001]
Epoch[0/5](200/237)||training loss 0.06814 || training accuracy 91.25% || lr [0.0001]
Epoch[0/5](220/237)||training loss 0.05989 || training accuracy 91.88% || lr [0.0001]
Calculating validation results
New best model for val accuracy! s

Train_loss,0.02349
Train_acc,0.97266
_runtime,1129
_timestamp,1617861858
_step,59
Valid_loss,0.37625
Valid_acc,0.77566


Train_loss,█▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train_acc,▁▄▅▆▆▆▆▆▇▇█████████████████████████▇███▇
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Valid_loss,▁▄▃▆█
Valid_acc,▄▆▇█▁


Epoch[0/5](20/237)||training loss 0.8465 || training accuracy 60.00% || lr [0.0001]
Epoch[0/5](40/237)||training loss 0.3368 || training accuracy 79.30% || lr [0.0001]
Epoch[0/5](60/237)||training loss 0.1947 || training accuracy 82.27% || lr [0.0001]
Epoch[0/5](80/237)||training loss 0.1438 || training accuracy 83.52% || lr [0.0001]
Epoch[0/5](100/237)||training loss 0.141 || training accuracy 83.36% || lr [0.0001]
Epoch[0/5](120/237)||training loss 0.1095 || training accuracy 87.81% || lr [0.0001]
Epoch[0/5](140/237)||training loss 0.104 || training accuracy 87.66% || lr [0.0001]
Epoch[0/5](160/237)||training loss 0.08892 || training accuracy 90.78% || lr [0.0001]
Epoch[0/5](180/237)||training loss 0.07328 || training accuracy 91.17% || lr [0.0001]
Epoch[0/5](200/237)||training loss 0.06273 || training accuracy 91.72% || lr [0.0001]
Epoch[0/5](220/237)||training loss 0.05354 || training accuracy 92.97% || lr [0.0001]
Calculating validation results
New best model for val accuracy! sav

...

Train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train_acc,▁▄▅▅▅▆▆▇▇▇▇█████████████████████████▇▇██
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Valid_loss,▁▅▄██
Valid_acc,▁▆██▁


Epoch[0/5](20/237)||training loss 0.8081 || training accuracy 63.44% || lr [0.0001]
Epoch[0/5](40/237)||training loss 0.3223 || training accuracy 79.30% || lr [0.0001]
Epoch[0/5](60/237)||training loss 0.1855 || training accuracy 83.05% || lr [0.0001]
Epoch[0/5](80/237)||training loss 0.1551 || training accuracy 84.69% || lr [0.0001]
Epoch[0/5](100/237)||training loss 0.1355 || training accuracy 84.61% || lr [0.0001]
Epoch[0/5](120/237)||training loss 0.1084 || training accuracy 86.33% || lr [0.0001]
Epoch[0/5](140/237)||training loss 0.08914 || training accuracy 89.06% || lr [0.0001]
Epoch[0/5](160/237)||training loss 0.08062 || training accuracy 89.61% || lr [0.0001]
Epoch[0/5](180/237)||training loss 0.07808 || training accuracy 90.62% || lr [0.0001]
Epoch[0/5](200/237)||training loss 0.07743 || training accuracy 90.47% || lr [0.0001]
Epoch[0/5](220/237)||training loss 0.07902 || training accuracy 90.47% || lr [0.0001]
Calculating validation results
New best model for val accuracy! 

Train_loss,0.02145
Train_acc,0.97266
_runtime,1138
_timestamp,1617865287
_step,59
Valid_loss,0.30389
Valid_acc,0.78407


Train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train_acc,▁▄▅▅▅▆▆▆▇▇▇█████████████████████████▇▇▇█
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Valid_loss,▁▆▄▇█
Valid_acc,▆██▆▁


Epoch[0/5](20/237)||training loss 0.8522 || training accuracy 61.48% || lr [0.0001]
Epoch[0/5](40/237)||training loss 0.337 || training accuracy 79.53% || lr [0.0001]
Epoch[0/5](60/237)||training loss 0.1816 || training accuracy 83.75% || lr [0.0001]
Epoch[0/5](80/237)||training loss 0.1345 || training accuracy 86.41% || lr [0.0001]
Epoch[0/5](100/237)||training loss 0.1268 || training accuracy 86.72% || lr [0.0001]
Epoch[0/5](120/237)||training loss 0.1085 || training accuracy 88.28% || lr [0.0001]
Epoch[0/5](140/237)||training loss 0.1021 || training accuracy 88.28% || lr [0.0001]
Epoch[0/5](160/237)||training loss 0.07166 || training accuracy 90.78% || lr [0.0001]
Epoch[0/5](180/237)||training loss 0.07461 || training accuracy 91.09% || lr [0.0001]
Epoch[0/5](200/237)||training loss 0.0718 || training accuracy 93.05% || lr [0.0001]
Epoch[0/5](220/237)||training loss 0.05712 || training accuracy 92.66% || lr [0.0001]
Calculating validation results
New best model for val accuracy! sav

In [14]:
from IPython.display import Audio
Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg", autoplay=True)

In [15]:
criterion = FocalLoss()

In [16]:
model = timm.create_model(model_name,pretrained = True)
# last_num  = model.classifier.in_features
# model.classifier = nn.Linear(last_num,3,bias=True)
model.reset_classifier(3)
model.to(device)
optimizer = AdamP(model.parameters(), lr = lr, weight_decay = 5e-4)

In [17]:
scheduler = CosineAnnealingLR(optimizer, T_max = 2, eta_min = 0.)

In [18]:
# ### Configuration
# data_dir = '/opt/ml/input/data/train'
# img_dir = f'{data_dir}/images'
# df_path = f'{data_dir}/train.csv'
# df = pd.read_csv(df_path)

In [19]:
# # 한 디렉토리 안에 여러 확장자가 들어있는 경우에 맞게 수정
# def get_ext(img_dir,img_id):
#     filename = glob(os.path.join(img_dir,img_id)+'/*.*')
#     ext = list(map(lambda x : os.path.splitext(x)[-1].lower(), filename))
#     return ext


In [20]:
from albumentations import *
from albumentations.pytorch import ToTensorV2

# def get_transforms(need = ('train','val'), img_size = (512,384), mean = (0.560, .524,.501), std = (.233,.243,.247)):
def get_transforms(need = ('train','val'), img_size = (300,300), mean = (0.560, .524,.501), std = (.233,.243,.247)):
    transformations = {}
    if 'train' in need:
        transformations['train'] = Compose([
            CenterCrop(500,300, p = 1.),
            HorizontalFlip(p = 0.5),
            RandomBrightnessContrast(brightness_limit = (-0.1,0.1), contrast_limit = (-0.1,0.1), p = 0.5),
            Normalize(mean=mean, std=std, max_pixel_value = 255., p = 1.),
            ToTensorV2(p = 1.),
        ], p = 1.)
        
    if 'val' in need :
        transformations['val'] = Compose([
            CenterCrop(500,300, p = 1.),
            Normalize(mean = mean, std=std, max_pixel_value = 255. , p = 1.),
            ToTensorV2(p = 1.),
        ], p = 1.)
    return transformations

In [21]:

class MaskBaseDataset(data.Dataset):
    num_classes = 3*2*3

    class MaskLabels:
        mask = 0
        incorrect = 1
        normal = 2

    class GenderLabels:
        male = 0
        female = 1

    class AgeGroup:
        map_label = lambda x: 0 if int(x) < 30 else 1 if int(x) < 58 else 2

    _file_names = {
        "mask1":MaskLabels.mask,
        "mask2":MaskLabels.mask,
        "mask3":MaskLabels.mask,
        "mask4":MaskLabels.mask,
        "mask1":MaskLabels.mask,
        "incorrect_mask":MaskLabels.incorrect,
        "normal": MaskLabels.normal
    }

    image_paths = []
    mask_labels = []
    gender_labels = []
    age_labels = []
    # 위와 같이 저장되는 값이 init에 저장될 때와 안될 때의 차이?

    def __init__(self, data_dir, mean = (.548, .504, .479), std = (.237, .247, .246), val_ratio = 0.2):
        self.data_dir = data_dir
        self.mean = mean
        self.std = std
        self.val_ratio = val_ratio

        self.transform = None
        self.setup()
        self.calc_statistics()


    def setup(self):
        profiles = os.listdir(self.data_dir)# 이걸 glob으로 했던 것 같은데 확인
        for profile in profiles:
            if profile.startswith('.'): # glob으로 안하고 예외처리를 하셨네/ glob os.listdir 차이 확인
                continue

            img_folder = os.path.join(self.data_dir, profile)
            for file_name in os.listdir(img_folder):
                _file_name, ext = os.path.splitext(file_name)
                if _file_name not in self._file_names: # .으로 시작하는 파일 및 invalid한 파일 무시(??)ㅏ
                    continue

                img_path = os.path.join(self.data_dir, profile, file_name) 
                # (resized_data, 00003_male_Asian_54, mask1.jpg) _file_name이 아니라 file_name 호출
                mask_label = self._file_names[_file_name]

                id, gender, race, age = profile.split('_')
                gender_label = getattr(self.GenderLabels, gender)
                age_label = self.AgeGroup.map_label(age)

                self.image_paths.append(img_path)
                self.mask_labels.append(mask_label)
                self.gender_labels.append(gender_label)
                self.age_labels.append(age_label)

    def calc_statistics(self):
        has_statistics = self.mean is not None and self.std is not None
        if not has_statistics:
            print("[Warning] Calculating statistics... It can takes huge amounts of time depending on your CPU machine")
            sums = []
            squared = []
            for image_path in self.image_paths[:3000]: # 왜 3000까지만?
                image = np.array(Image.open(image_path)).astype(np.int32)
                sums.append(image.mean(axis = (0,1))) # axis에 값 두개 주는것이 어떤 의미인가
                squared.append((image**2).mean(axis = (0,1)))

            self.mean = np.mean(sums, axis = 0)/ 255
            self.std = (np.mean(squared, axis = 0) - self.mean**2) ** 0.5 / 255

    def set_transform(self, transform):
        self.transfrom = transform

    def __getitem__(self, index):
        image = self.read_image(index)
        mask_label = self.get_mask_label(index)
        gender_label = self.get_gender_label(index)
        age_label = self.get_age_label(index)
        multi_class_label = self.encode_multi_class(mask_label, gender_label, age_label)

        image_transform = self.transform(image = np.array(image))['image']
#         image_transform = self.transform(image=np.array(image))['image']
        
        return image_transform , multi_class_label

    def __len__(self):
        return len(self.image_paths)

    def get_mask_label(self, index):
        return self.mask_labels[index]

    def get_gender_label(self, index):
        return self.gender_labels[index]

    def get_age_label(self, index):
        return self.age_labels[index]

    def read_image(self,index):
        image_path = self.image_paths[index]
        return Image.open(image_path)

    @staticmethod
    def encode_multi_class(mask_label, gender_label, age_label):
        return mask_label # * 6 gender_label * 3 + age_label

    @staticmethod
    def decode_multi_class(multi_class_label):
        mask_label = (multi_class_label//6) % 3
        gender_label  = (multi_class_label//3) % 2
        age_label = multi_class_label % 3
        return mask_label, gender_label, age_label

    @staticmethod
    def denormalize_image(image,mean,std):
        img_cp = image.copy()
        img_cp *= std
        img_cp += mean
        img_cp *= 255.0
        img_cp = np.clip(img_cp,0,255).astype(np.uint8) # np.clip
        return img_cp

    def split_dataset(self):
        n_val = int(len(self) * self.val_ratio)
        n_train = len(self) - n_val
        train_set, val_set = torch.utils.data.random_split(self, [n_train, n_val])
        return train_set, val_set

    
class MaskSplitByProfileDataset(MaskBaseDataset):
    """
        train/val 나누는 기준을 이미지에 대해서 random이 아닌
        사람(profile)을 기준으로 나눕니다.
        구현은 val_ratio에 맞게 train/ val 나누는 것을 이미지 전체가 아닌 사람(profile)에 대해서 진행하여 indexing을 합니다.
        이후 'split_dataset'에서 index에 맞게 Subset으로 dataset을 분기합니다.
    """
    
    def __init__(self, data_dir, mean = (.548, .504, .479), std = (.237, .247, .246), val_ratio =  0.2):
        self.indices = defaultdict(list)
        super().__init__(data_dir, mean, std, val_ratio)

    @staticmethod
    def _split_profile(profiles, val_ratio):
        length = len(profiles)
        n_val = int(length * val_ratio)

        val_indices = set(random.sample(range(length), k = n_val))
        train_indices = set(range(length)) - val_indices

        return {
            "train" : train_indices,
            "val" : val_indices
        }

    def setup(self):
        profiles = os.listdir(self.data_dir)
        profiles = [profile for profile in profiles if not profile.startswith('.')]
        split_profiles = self._split_profile(profiles, self.val_ratio)

        cnt = 0
        for phase, indices  in split_profiles.items():
            for _idx in indices:
                profile = profiles[_idx]
                img_folder = os.path.join(self.data_dir, profile)
                for file_name in os.listdir(img_folder):
                    _file_name, ext = os.path.splitext(file_name)
                    if _file_name not in self._file_names:
                        continue

                    img_path = os.path.join(self.data_dir, profile, file_name)
                    mask_label = self._file_names[_file_name]

                    id, gender, race, age = profile.split('_')
                    gender_label = getattr(self.GenderLabels, gender)
                    age_label = self.AgeGroup.map_label(age)

                    self.image_paths.append(img_path)
                    self.mask_labels.append(mask_label)
                    self.gender_labels.append(gender_label)
                    self.age_labels.append(age_label)

                    self.indices[phase].append(cnt)
                    cnt += 1
    def set_transform(self, transform):
        self.transform = transform
        
    def split_dataset(self):
        return [Subset(self, indices) for phase, indices in self.indices.items()]


In [22]:
mean = (0.560, .524,.501)
std = (.233,.243,.247)

transform = get_transforms(mean = mean, std = std)
    
# dataset = MaskBaseDataset(img_dir = img_dir)

dataset = MaskSplitByProfileDataset(data_dir = img_dir)
dataset.setup()
train_dataset, val_dataset = dataset.split_dataset()

train_dataset.dataset.set_transform(transform['train'])
val_dataset.dataset.set_transform(transform['val'])

train_loader = data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True
)

val_loader = data.DataLoader(
    val_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = False
)

patience = 10
counter = 0

accumulation_steps = 2

In [23]:
os.makedirs(os.path.join(os.getcwd(),'results', model_name), exist_ok = True)

counter =0 
best_val_acc = 0
best_val_loss = np.inf
for epoch in range(num_epochs):
    #train loop
    model.train()
    loss_value = 0
    matches = 0
    for idx, train_batch in enumerate(train_loader):
        inputs, labels= train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = model(inputs)
        preds = torch.argmax(outs, dim = -1)
        loss = criterion(outs, labels)
        
        loss.backward()
        
        if (idx+1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        if (idx+1) % train_log_interval == 0:
            train_loss = loss_value/ train_log_interval
            train_acc = matches / batch_size/ train_log_interval
            current_lr = scheduler.get_last_lr()
            wandb.log({
                    "Train_loss": train_loss,
                    "Train_acc": train_acc
                })
            print(
                f'Epoch[{epoch}/{num_epochs}]({idx+1}/{len(train_loader)})||'
                f'training loss {train_loss:4.4} || training accuracy {train_acc:4.2%} || lr {current_lr}'
            )
            
            loss_value = 0
            matches = 0
    scheduler.step()
    
    #val loop
    with torch.no_grad():
        print("Calculating validation results")
        model.eval()
        val_loss_items = []
        val_acc_items= []
        for val_batch in val_loader:
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = model(inputs)
            preds = torch.argmax(outs, dim = -1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
        
        val_loss = np.sum(val_loss_items)/ len(val_loader)
        val_acc  = np.sum(val_acc_items)/ len(val_dataset)
        wandb.log({
                "Valid_loss": val_loss,
                "Valid_acc": val_acc
            })
        if val_loss < best_val_loss:
            best_val_loss = val_loss
        if val_acc > best_val_acc:
            print('New best model for val accuracy! saving the model..')
            now = time.localtime()
            torch.save(model.state_dict(), f'results/{model_name}/{epoch:03}_accuracy_{val_acc:4.2%}_{now.tm_hour+9}:{now.tm_min}_mask.ckpt')
            best_val_acc = val_acc
            counter = 0
        else:
            counter += 1
        
        if counter > patience:
            print('Early stopping...')
            break
            
        print(
            f'[Val] acc : {val_acc:4.2%}, loss : {val_loss:4.2} ||'
            f'best acc : {best_val_acc:4.2%}, best loss: {best_val_loss:4.2}'
        )

Epoch[0/5](20/405)||training loss 0.1875 || training accuracy 87.89% || lr [0.0001]
Epoch[0/5](40/405)||training loss 0.01453 || training accuracy 99.22% || lr [0.0001]
Epoch[0/5](60/405)||training loss 0.009767 || training accuracy 99.06% || lr [0.0001]
Epoch[0/5](80/405)||training loss 0.009246 || training accuracy 99.69% || lr [0.0001]
Epoch[0/5](100/405)||training loss 0.002587 || training accuracy 99.77% || lr [0.0001]
Epoch[0/5](120/405)||training loss 0.007518 || training accuracy 99.38% || lr [0.0001]
Epoch[0/5](140/405)||training loss 0.004138 || training accuracy 99.69% || lr [0.0001]
Epoch[0/5](160/405)||training loss 0.00396 || training accuracy 99.45% || lr [0.0001]
Epoch[0/5](180/405)||training loss 0.0038 || training accuracy 99.69% || lr [0.0001]
Epoch[0/5](200/405)||training loss 0.001672 || training accuracy 99.84% || lr [0.0001]
Epoch[0/5](220/405)||training loss 0.002754 || training accuracy 99.77% || lr [0.0001]
Epoch[0/5](240/405)||training loss 0.01049 || trainin

KeyboardInterrupt: 

In [ ]:
from IPython.display import Audio
Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg", autoplay=True)